This is a work-in-progress reproduction of the [Biological Network Exploration with Cytoscape 3](https://pubmed.ncbi.nlm.nih.gov/25199793/) Basic Protocol 1, which loads an s. cervesiae network, filters out unneeded nodes, lays out the resulting network, and then creates a dendogram display.

While much of it works, there are compromises, mainly due to Cytoscape features that aren't at full strength yet.

# Setup data files, py4cytoscape and Cytoscape connection
---
**NOTE: To run this notebook, you must manually start Cytoscape first -- don't proceed until you have started Cytoscape.**

---
## Setup: Fetch latest py4cytoscape




In [15]:
import py4cytoscape as p4c

---
## Setup: Sanity test to verify Cytoscape connection


By now, the connection to Cytoscape should be up and available. To verify this, try a simple operation that doesn't alter the state of Cytoscape.

In [2]:
p4c.cytoscape_version_info()


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.9.1',
 'automationAPIVersion': '1.4.0',
 'py4cytoscapeVersion': '1.3.0'}

---
## Setup: Notebook data files

Create the 'output' directory, which will be used to store files uploaded from Cytoscape.

This is a good place to prepare any other system resources that might be needed by downstream Notebook cells.

Pro Tip: The "!" commands in this cell are passed to the host operating system. In this example, they're correct for a Windows host. Different commands would be appropriate for a Linux or Mac host.


In [3]:
!del /s/q/f output
!rmdir output
!mkdir output
!dir output
OUTPUT_DIR = 'output/'

 Volume in drive C has no label.
 Volume Serial Number is 50EF-8726

 Directory of C:\Users\CyDeveloper\PycharmProjects\py4cytoscape\tests\Notebooks\output

05/13/2022  05:27 PM    <DIR>          .
05/13/2022  05:27 PM    <DIR>          ..
               0 File(s)              0 bytes
               2 Dir(s)   1,957,814,272 bytes free


---
## Setup: Import source data files

The network and annotation files are in a Dropbox folder, and this cell downloads them into the current Cytoscape directory, which also holds this Jupyter Notebook.

The files could just as well have been on any cloud resource, including Google Drive, Github, Microsoft OneDrive or a private web site. Note that in this case, the network file was so large that it could not be saved on GitHub, so Dropbox was a handy alternative.

*An alternative would be to loading them into the Cytoscape workstation file system using the Notebook "!" commands (e.g., !wget). That's out of the scope of this tutorial, though there's an example in the Merging cell below.*

---
Note that this cell uses the import_file_from_url() function to load resources from cloud storage.
This function is appropriate for Notebooks running on the same workstation as Cytoscape, but
not for Notebooks running on a remote server. On a remote server, the Notebook's file system is not the same as the Cytoscape workstation's file system, and Sandbox functions (https://py4cytoscape.readthedocs.io/en/latest/concepts.html#sandboxing) should be used instead.

In [4]:
res_mitab = p4c.import_file_from_url("https://www.dropbox.com/s/8wc8o897tsxewt1/BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab?dl=0", "BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab")
print(f'Network file BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab has {res_mitab["fileByteCount"]} bytes')

Network file BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab has 166981992 bytes


# Load the s. cerevisiae MITAB network into Cytoscape

Note that the import_network_from_file function (incorrectly) throws an exception, so we explicitly ignore the exception.

**Note**: Once CYTOSCAPE-12772 is fixed, we can remove the try-block in this cell.

In [5]:
from requests import HTTPError
p4c.close_session(False)

try:
  p4c.import_network_from_file('BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab')
except:  
  pass
if p4c.get_network_count() != 1:
  raise Exception('Failed to load network')
net_suid = p4c.get_network_suid()
print(f'Network identifier: {net_suid}')



Network identifier: 8881228


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/C:/Users/CyDeveloper/CytoscapeConfiguration/3/framework-cytoscape.log'}


# Merge the gene expression data into the node table

For Cytoscape 3.9.0 and later, call Cytoscape to merge the gene expression data into the node attribute table. 

For pre-Cytoscape 3.9.0, do most of the work in Pandas and then import the dataframe into the node attribute table. Explicitly set the Gene ID as a string even though it's originally parsed as a number. To Cytoscape, the string will be comparable to the 'name' column already in the BIOGRID network. The Gene ID column in the dataframe is matched to the network's name column.

Pro Tip: The wget and mv commands work on a Jupyter system on a Linux host. You may have to choose different commands for a Windows host.

In [6]:
if p4c.check_supported_versions(cytoscape='3.9') is None:
  # Load file directly into Sandbox so Cytoscape can import it
  res_soft = p4c.import_file_from_url("https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0", "GDS112_full.soft")
  print(f'Annotation file GDS112_full.soft has {res_soft["fileByteCount"]} bytes')

  res = p4c.load_table_data_from_file('GDS112_full.soft', start_load_row=83, data_key_column_index=10, delimiters='\t')
  print(f'Load result contains table identifiers: {res["mappedTables"]}')
else:
  # Load file into Notebook file system so Python can import it, tweak it, and download to Cytoscape
  !wget -q --no-check-certificate https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0
  !mv GDS112_full.soft?dl=0 GDS112_full.soft

  import pandas as df
  GDS112_full = df.read_csv('GDS112_full.soft', skiprows=82, sep='\t')
  GDS112_full.dropna(subset=['Gene ID'], inplace=True)
  GDS112_full['Gene ID'] = df.to_numeric(GDS112_full['Gene ID'], downcast='integer')
  GDS112_full = GDS112_full.astype({'Gene ID': 'string'})
  print(GDS112_full.dtypes)
  print(GDS112_full)
  p4c.load_table_data(GDS112_full, data_key_column='Gene ID')

  import os
  os.remove('GDS112_full.soft')


Annotation file GDS112_full.soft has 5536880 bytes
Load result contains table identifiers: [8881199, 8881237]


# Create a filter to remove nodes having no Gene Symbol

In [7]:
res = p4c.create_column_filter('SymbolOK', 'Gene symbol', '[A-Z0-9]*', 'REGEX')
print(f'Nodes selected: {len(res["nodes"])}')

No edges selected.
Nodes selected: 5516


# Create a subnetwork containing only named nodes

This could take several minutes.

At the end, you should see a view containing all nodes laid out. 

If you see only a single rectangle, it could be that your Cytoscape is set to operate with a small stack size. To increase the stack:

1. terminate Cytoscape

2. a) upgrade Cytoscape to 3.9.0 or later 

  ... or b) use a text editor to add -Xss5M to the cytoscape.vmoptions file in your Cytoscape program directory

3. restart Cytoscape

4. re-run this workflow

In [8]:
new_suid = p4c.create_subnetwork()
print(f'New network identifier: {new_suid}')

New network identifier: 10269126


# Get rid of the original network, which isn't needed anymore

In [9]:
p4c.delete_network(net_suid)
net_suid = new_suid

# Install clusterMaker2 if it hasn't already been installed

In [10]:
p4c.install_app('clusterMaker2')

{}


{}

# Create the hierarchical clustering and dendogram

This returns a large data structure that describes the dendogram.

It also creates a dendogram window that's designed for GUI manipulation. It's unclear this can be controlled or used by automation calls.

**Note:** Having the dendogram is important, and so is having the data that created it. When CSD-420 is addressed, it will be possible to snapshot the dendogram and perform other operations with it.

In [11]:
res = p4c.commands_post('cluster hierarchical showUI=true clusterAttributes=false nodeAttributeList="GSM1029,GSM1030,GSM1032,GSM1033,GSM1034"')
print(f'Dendogram tree: {res}')

Dendogram tree: [{'nodeOrder': [{'nodeName': '850532', 'suid': 8903726}, {'nodeName': '851208', 'suid': 8882102}, {'nodeName': '852107', 'suid': 8887022}, {'nodeName': '852854', 'suid': 9216422}, {'nodeName': '850695', 'suid': 8884142}, {'nodeName': '851125', 'suid': 8883962}, {'nodeName': '850422', 'suid': 8883971}, {'nodeName': '852549', 'suid': 8889488}, {'nodeName': '851366', 'suid': 8937536}, {'nodeName': '854469', 'suid': 8953226}, {'nodeName': '852704', 'suid': 8894582}, {'nodeName': '853534', 'suid': 8903417}, {'nodeName': '850820', 'suid': 8883964}, {'nodeName': '855473', 'suid': 8885153}, {'nodeName': '853762', 'suid': 8894639}, {'nodeName': '853682', 'suid': 8882084}, {'nodeName': '850894', 'suid': 8884286}, {'nodeName': '856063', 'suid': 8882132}, {'nodeName': '855730', 'suid': 8882204}, {'nodeName': '854116', 'suid': 8885966}, {'nodeName': '850872', 'suid': 8881711}, {'nodeName': '850893', 'suid': 8885984}, {'nodeName': '854381', 'suid': 8882635}, {'nodeName': '855591', 's

# Use BiNGO for enrichment analysis

The BiNGO app doesn't have automation entrypoints, so this analysis isn't possible right now. Is there a different app that can do this?

**NOTE:** We need CSD-421 fixed because we don't have any analysis right now, which is very important.

In [12]:
p4c.node_name_to_node_suid('851303')

[8920322]

In [13]:
print(res[0]['nodeOrder'])
print(res[0]['nodeTree'])
node_suid = p4c.node_name_to_node_suid('851303')[0] # Get SUID for HBT1


[{'nodeName': '850532', 'suid': 8903726}, {'nodeName': '851208', 'suid': 8882102}, {'nodeName': '852107', 'suid': 8887022}, {'nodeName': '852854', 'suid': 9216422}, {'nodeName': '850695', 'suid': 8884142}, {'nodeName': '851125', 'suid': 8883962}, {'nodeName': '850422', 'suid': 8883971}, {'nodeName': '852549', 'suid': 8889488}, {'nodeName': '851366', 'suid': 8937536}, {'nodeName': '854469', 'suid': 8953226}, {'nodeName': '852704', 'suid': 8894582}, {'nodeName': '853534', 'suid': 8903417}, {'nodeName': '850820', 'suid': 8883964}, {'nodeName': '855473', 'suid': 8885153}, {'nodeName': '853762', 'suid': 8894639}, {'nodeName': '853682', 'suid': 8882084}, {'nodeName': '850894', 'suid': 8884286}, {'nodeName': '856063', 'suid': 8882132}, {'nodeName': '855730', 'suid': 8882204}, {'nodeName': '854116', 'suid': 8885966}, {'nodeName': '850872', 'suid': 8881711}, {'nodeName': '850893', 'suid': 8885984}, {'nodeName': '854381', 'suid': 8882635}, {'nodeName': '855591', 'suid': 8889521}, {'nodeName': '8

In [14]:
import parse_dendogram as pde
node_order = res[0]['nodeOrder']
node_tree = res[0]['nodeTree']
node_bag = pde.create_node_bag(node_order, node_tree)
select_nodes = pde.find_node_set(node_suid, 85, node_order, node_bag)
select_nodes = list(select_nodes)
print(select_nodes)
p4c.select_nodes(select_nodes, preserve_current_selection=False)

[9005057, 8920322, 9016328, 8923657, 8924941, 9320210, 9011219, 8952596, 9872405, 8976410, 9214748, 9102365, 8970785, 8966945, 8896805, 9084455, 8887595, 8885036, 8924204, 9339440, 8913713, 8964146, 8923187, 8913719, 8924216, 9069632, 8904515, 8923466, 8902730, 8921677, 8887631, 8903255, 9074264, 8901719, 8925023, 8882027, 8901740, 8887660, 9008495, 8882288, 9284723, 8904824, 8993915, 9022331, 8882047, 8944262, 8901515, 8915084, 9073805, 8904590, 8974736, 8904592, 8924306, 8908442, 8901788, 8900510, 9011615, 8978342, 9213611, 8886956, 9213356, 8906678, 9022904, 8893880, 8924090, 9033401, 9073853, 8900801, 8901317, 9077957, 9008585, 8883401, 8887499, 8971214, 8989139, 8956628, 8895446, 8900822, 8917976, 9654749, 8987885, 8894702, 8889839, 9313523, 8988668]


{'nodes': [8988668,
  8993915,
  9022331,
  8882288,
  8889839,
  9008495,
  9284723,
  9313523,
  8904824,
  9016328,
  8923657,
  9073805,
  8915084,
  8924941,
  8882047,
  8904590,
  8901515,
  8920322,
  9005057,
  8944262,
  9214748,
  8976410,
  9102365,
  8904592,
  8900510,
  8974736,
  8901788,
  8924306,
  8908442,
  8952596,
  9320210,
  9011219,
  9872405,
  9213611,
  8896805,
  9213356,
  9084455,
  8924204,
  9011615,
  8966945,
  8970785,
  8886956,
  8885036,
  8887595,
  8978342,
  9022904,
  8906678,
  8913719,
  8924216,
  9033401,
  8924090,
  9073853,
  8913713,
  8964146,
  8923187,
  8893880,
  9339440,
  8901317,
  8904515,
  9008585,
  8923466,
  8900801,
  8921677,
  8971214,
  8887499,
  9069632,
  8902730,
  9077957,
  8883401,
  8895446,
  8900822,
  8917976,
  9074264,
  8887631,
  8956628,
  8989139,
  8903255,
  8901719,
  8987885,
  8925023,
  8894702,
  8887660,
  8901740,
  8882027,
  9654749],
 'edges': []}